In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import os

import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, concatenate
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow_addons as tfa

In [ ]:
#Variables
image_size = 256
in_channel_spec = 9
in_channel_tool = 3
num_classes = 3
batch_size = 1

In [ ]:
#Functions

#Convert images to numpy array

def read_tools(file_paths, image_size, channels):
  images = []
  
  for file_path in file_paths:
    img = cv2.imread(file_path)
    res = cv2.resize(img, dsize=(image_size, image_size), interpolation=cv2.INTER_CUBIC)
    images.append(res)
  
  images = np.asarray(images, dtype=np.float32)
  
  # normalize
  images = images / np.max(images)
  
  # reshape to match Keras expectaions
  images = images.reshape(images.shape[0], image_size, image_size, channels)

  return images

def read_specs(file_paths_x, file_paths_y, file_paths_z, image_size, channels):
  images = []
  
  for i in range(file_paths_x.size):
    img_x = cv2.imread(file_paths_x[i])
    res_x = cv2.resize(img_x, dsize=(image_size, image_size), interpolation=cv2.INTER_CUBIC)
    images.append(res_x)
    img_y = cv2.imread(file_paths_y[i])
    res_y = cv2.resize(img_y, dsize=(image_size, image_size), interpolation=cv2.INTER_CUBIC)
    images.append(res_y)
    img_z = cv2.imread(file_paths_z[i])
    res_z = cv2.resize(img_z, dsize=(image_size, image_size), interpolation=cv2.INTER_CUBIC)
    images.append(res_z)
  
  images = np.asarray(images, dtype=np.float32)
  images = images / np.max(images)
  images = images.reshape(file_paths_x.shape[0], image_size, image_size, channels)

  return images


#Patch dataset

def generate_datasets(images, labels, is_train=False):
    dataset = images
    dataset = dataset.batch(batch_size)
    if is_train:
        dataset = dataset.map(
            lambda x, y: (data_augmentation(x), y), num_parallel_calls=auto
            
        )
    return dataset.prefetch(auto)

In [ ]:
#Dataset acquisition
test_df = pd.read_csv('../Data/Labels/test.csv', index_col=0)

test_df['tool'] = test_df.index.map(lambda id: f'../Data/Datasets/tool/{id}.jpg')
test_df['spec_x'] = test_df.index.map(lambda id: f'../Data/Datasets/specX/{id}.png')
test_df['spec_y'] = test_df.index.map(lambda id: f'../Data/Datasets/specY/{id}.png')
test_df['spec_z'] = test_df.index.map(lambda id: f'../Data/Datasets/specZ/{id}.png')

#Read tool images and spectrograms and convert them to NumPy array
x_test_tool = read_tools(test_df.tool.values, image_size, in_channel_tool)
x_test_spec = read_specs(test_df.spec_x.values,test_df.spec_y.values,test_df.spec_z.values, image_size, in_channel_spec)

In [ ]:
#Bring the labels to an acceptable form
labels_test = test_df.tool_label.values - 1

labels_test = tf.keras.utils.to_categorical(
    labels_test, num_classes, dtype='float32')

#Create tensorflow datasets objects and add patch embedding to train dataset
dataset_test = tf.data.Dataset.from_tensor_slices(({'input_2_tool': x_test_tool, 'input_2_spec': x_test_spec}, labels_test))

auto = tf.data.AUTOTUNE
inputs1 = k.Input((image_size, image_size, in_channel_tool))
inputs2 = k.Input((image_size, image_size, in_channel_spec))
tool_crop = k.layers.RandomCrop(image_size, image_size)(inputs1)
spec_crop = k.layers.RandomCrop(image_size, image_size)(inputs2)
tool_crop = k.layers.RandomFlip("horizontal")(tool_crop)
spec_crop = k.layers.RandomFlip("horizontal")(spec_crop)

data_augmentation = k.Model(
    inputs={'input_2_tool': inputs1, 'input_2_spec': inputs2},
    outputs={'input_2_tool':tool_crop, 'input_2_spec':spec_crop},
)

dataset_test = generate_datasets(dataset_test, labels_test)

In [ ]:
#Load multimodal network
model = k.models.load_model(
    "../models/multimodal_aug_tool.h5", compile=True)

In [ ]:
#Evaluate multimodal network
result_eval = model.evaluate(dataset_test)
print(result_eval)

In [ ]:
#Classify test datset
result_predict = model.predict(dataset_test)
result_predict = np.argmax(result_predict, axis=1)
print(result_predict)